In [1]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
# !pip install joblib

In [3]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
# Keys to columns
# 4 fp flags (false positive flags)
# koi_period (orbital period in days)
# koi_time0bk (transit epoch)
# koi_impact (impact parameter)
# koi_duration (transit duration in hours)
# koi_depth (transit depth in ppm)
# koi_prad (planetary radius in Earth radii)
# koi_teq (equilibrium temperature in Kelvins)
# koi_insol (insolation flux in Earth flux)
# koi_model_snr (transit signal-to-noise)
# koi_tce_plnt_num (TCE planet number)
# koi_steff (stellar effective temperature in Kelvins)
# koi_slogg (stellar surface gravity logarithm)
# koi_srad (stellar radius)
# ra (RA - decimal degrees)
# dec (Dec - decimal degrees)
# koi_kepmag (Kepler-band in mag)

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_duration',
                       'koi_depth', 'koi_prad', 'koi_steff', 'koi_slogg', 'koi_srad']]

# Note: Do not select the error margins 
# Items of interest:
# koi_disposition (confirmed planet or not)
# koi_period (orbital period in days)
# koi_time0bk (transit epoch) - How does transit epoch differ from transit duration (below)? 
# koi_impact (impact parameter) - need more information.  What is this?  
# koi_duration (transit duration in hours)
# koi_depth (transit depth in ppm)
# koi_prad (planetary radius in Earth radii) - Hypothesis:  Smaller objects may be asteroids, not planets.
# koi_steff (stellar effective temperature in Kelvins)
# koi_slogg (stellar surface gravity logarithm) - Hypothesis:  Stronger surface gravity of stars may keep objects in orbit - may indicate planethood.
# koi_srad (stellar radius) - Hypothesis:  Smaller stars may be less likely to have planets. 

selected_features = selected_features.rename(columns = {
    'koi_disposition': "Planetary Status", 
    'koi_period': "Orbit period (d)", 
    'koi_time0bk': "Transit epoch", 
    'koi_duration': "Transit duration (h)",
    'koi_depth': "Transit depth", 
    "koi_prad": "Planet radius",
    'koi_steff': "Stellar temp (K)", 
    'koi_slogg': "Stellar gravity", 
    'koi_srad': "Stellar radius"
})

selected_features

,Planetary Status,Orbit period (d),Transit epoch,Transit duration (h),Transit depth,Planet radius,Stellar temp (K),Stellar gravity,Stellar radius
0,CONFIRMED,54.418383,162.513840,4.50700,874.8,2.83,5455,4.467,0.927
1,FALSE POSITIVE,19.899140,175.850252,1.78220,10829.0,14.60,5853,4.544,0.868
2,FALSE POSITIVE,1.736952,170.307565,2.40641,8079.2,33.46,5805,4.564,0.791
3,CONFIRMED,2.525592,171.595550,1.65450,603.3,2.75,6031,4.438,1.046
4,CONFIRMED,4.134435,172.979370,3.14020,686.0,2.77,6046,4.486,0.972
...,...,...,...,...,...,...,...,...,...
6986,FALSE POSITIVE,8.589871,132.016100,4.80600,87.7,1.11,5638,4.296,1.088
6987,FALSE POSITIVE,0.527699,131.705093,3.22210,1579.2,29.35,5638,4.529,0.903
6988,CANDIDATE,1.739849,133.001270,3.11400,48.5,0.72,6119,4.444,1.031
6989,FALSE POSITIVE,0.681402,132.181750,0.86500,103.6,1.07,6173,4.447,1.041


In [7]:
# Convert "Planetary Status" column to integer
selected_features = selected_features.replace({"CONFIRMED": "1.0", 
                                               "CANDIDATE": "0.0", 
                                               "FALSE POSITIVE": "-1.0"})

# selected_features.astype({"Planetary Status": "float64"}).dtypes
selected_features

,Planetary Status,Orbit period (d),Transit epoch,Transit duration (h),Transit depth,Planet radius,Stellar temp (K),Stellar gravity,Stellar radius
0,1.0,54.418383,162.513840,4.50700,874.8,2.83,5455,4.467,0.927
1,-1.0,19.899140,175.850252,1.78220,10829.0,14.60,5853,4.544,0.868
2,-1.0,1.736952,170.307565,2.40641,8079.2,33.46,5805,4.564,0.791
3,1.0,2.525592,171.595550,1.65450,603.3,2.75,6031,4.438,1.046
4,1.0,4.134435,172.979370,3.14020,686.0,2.77,6046,4.486,0.972
...,...,...,...,...,...,...,...,...,...
6986,-1.0,8.589871,132.016100,4.80600,87.7,1.11,5638,4.296,1.088
6987,-1.0,0.527699,131.705093,3.22210,1579.2,29.35,5638,4.529,0.903
6988,0.0,1.739849,133.001270,3.11400,48.5,0.72,6119,4.444,1.031
6989,-1.0,0.681402,132.181750,0.86500,103.6,1.07,6173,4.447,1.041


# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [9]:
target = selected_features["Planetary Status"]
# target_names = ["CONFIRMED", "CANDIDATE", "FALSE POSITIVE"]
target.head()

0     1.0
1    -1.0
2    -1.0
3     1.0
4     1.0
Name: Planetary Status, dtype: object

In [10]:
X = selected_features[["Planet radius", "Orbit period (d)", "Transit epoch", "Transit duration (h)",
                            "Transit depth", "Planet radius", "Stellar temp (K)", "Stellar gravity", "Stellar radius"]]
feature_names = X.columns
X.head()

,Planet radius,Orbit period (d),Transit epoch,Transit duration (h),Transit depth,Planet radius,Stellar temp (K),Stellar gravity,Stellar radius
0,2.83,54.418383,162.513840,4.50700,874.8,2.83,5455,4.467,0.927
1,14.60,19.899140,175.850252,1.78220,10829.0,14.60,5853,4.544,0.868
2,33.46,1.736952,170.307565,2.40641,8079.2,33.46,5805,4.564,0.791
3,2.75,2.525592,171.595550,1.65450,603.3,2.75,6031,4.438,1.046
4,2.77,4.134435,172.979370,3.14020,686.0,2.77,6046,4.486,0.972


In [11]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=42)

In [12]:
# Reshape the y values
y_train.shape
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)
print(y_train.shape)
print(y_test.shape)

(5243, 1)
(1748, 1)


In [13]:
# Convert y-labels from string to floats
# ct = LabelBinarizer()

# binarized_y_train = ct.fit_transform(y_train)
# binarized_y_test = ct.fit_transform(y_test)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [14]:
# Scale your data

X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

C:\Users\Rob-Desktop\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# Train the Model



In [ ]:
print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid1.best_params_)
print(grid1.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)